In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import dlib
import cv2

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./Models/shape_predictor_68_face_landmarks.dat')

with open('./Models/Handcrafted.pkl', "rb") as f:
    [clf2, sc2]= pickle.load(f)


In [2]:
def calc_dist(x0,y0,x1,y1):
    dist = (x0-x1)**2 + (y0-y1)**2
    dist = dist ** 0.5
    return dist

def PolygonArea(corners):
    n = len(corners) # of corners
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += corners[i][0] * corners[j][1]
        area -= corners[j][0] * corners[i][1]
    area = abs(area) / 2.0
    return area

def get_landmarks(im):
    rects = detector(im, 1)
    for (i, rect) in enumerate(rects):
            landmarks = predictor(im, rect)
            break
    return np.matrix([[p.x, p.y] for p in landmarks.parts()])

In [3]:
df4_neutral = pd.DataFrame(columns=('Athlete', 'Ground truth (BMI)', 'Prediction'))
Names=['LIM HYOJUN','DAVID GLEIRSCHER']
BMI=[22.67, 26.87]

for tra in range(2):
            img_path_1='./Testing_images/'+str(tra)+'.jpg'
            img1= cv2.imread(img_path_1, cv2.IMREAD_COLOR)
            landmark_list = get_landmarks(img1)
            
            columns=[]
            for each_land in range(0,68):
                columns.append('landx'+str(each_land))
                columns.append('landy'+str(each_land))
            bmi_data = pd.DataFrame(columns=columns)
            
            land_count = 0
            for each_land in landmark_list:
                one_land=each_land.tolist()
                bmi_data.loc[0, 'landx'+str(land_count)]=one_land[0][0]
                bmi_data.loc[0, 'landy'+str(land_count)]=one_land[0][1]
                land_count=land_count+1
                
            bmi_data['face_height3'] = calc_dist(bmi_data['landx27'],0,bmi_data['landx8'],bmi_data['landy8'])
            bmi_data['face_height2'] = calc_dist(bmi_data['landx27'],bmi_data['landy27'],bmi_data['landx66'],bmi_data['landy66'])
            bmi_data['face_width0'] = calc_dist(bmi_data['landx0'],bmi_data['landy0'],bmi_data['landx16'],bmi_data['landy16'])
            bmi_data['face_width4'] = calc_dist(bmi_data['landx4'],bmi_data['landy4'],bmi_data['landx12'],bmi_data['landy12'])
            bmi_data['eye_width1'] = calc_dist(bmi_data['landx36'],bmi_data['landy36'],bmi_data['landx45'],bmi_data['landy45'])
            bmi_data['eye_width2'] = calc_dist(bmi_data['landx39'],bmi_data['landy39'],bmi_data['landx42'],bmi_data['landy42'])
            bmi_data['face_height'] = calc_dist(bmi_data['landx27'],bmi_data['landy27'],bmi_data['landx8'],bmi_data['landy8'])

            corners=[(bmi_data['landx0'],bmi_data['landy0']), (bmi_data['landx4'],bmi_data['landy4']), (bmi_data['landx8'],bmi_data['landy8']), (bmi_data['landx12'],bmi_data['landy12']), (bmi_data['landx16'],bmi_data['landy16'])]
            seg_1=calc_dist(bmi_data['landx0'],bmi_data['landy0'],bmi_data['landx4'],bmi_data['landy4'])
            seg_2=calc_dist(bmi_data['landx8'],bmi_data['landy8'],bmi_data['landx4'],bmi_data['landy4'])
            seg_3=calc_dist(bmi_data['landx8'],bmi_data['landy8'],bmi_data['landx12'],bmi_data['landy12'])
            seg_4=calc_dist(bmi_data['landx16'],bmi_data['landy16'],bmi_data['landx12'],bmi_data['landy12'])

            mid_re_x=(bmi_data['landx43']+bmi_data['landx44'])/2.0
            mid_re_y=(bmi_data['landy43']+bmi_data['landy44'])/2.0                                                                                            
            mid_le_x=(bmi_data['landx37']+bmi_data['landx38'])/2.0
            mid_le_y=(bmi_data['landy37']+bmi_data['landy38'])/2.0

            d1=calc_dist(bmi_data['landx36'],bmi_data['landy36'],bmi_data['landx17'],bmi_data['landy17'])
            d2=calc_dist(bmi_data['landx19'],bmi_data['landy19'],mid_le_x,mid_le_y)                                                                                               
            d3=calc_dist(bmi_data['landx39'],bmi_data['landy39'],bmi_data['landx21'],bmi_data['landy21']) 
            d4=calc_dist(bmi_data['landx42'],bmi_data['landy42'],bmi_data['landx22'],bmi_data['landy22'])
            d5=calc_dist(bmi_data['landx24'],bmi_data['landy24'],mid_re_x,mid_re_y)                                                                                               
            d6=calc_dist(bmi_data['landx45'],bmi_data['landy45'],bmi_data['landx26'],bmi_data['landy26'])                                                                                              

            bmi_data['CJWR']=bmi_data['face_width0']/bmi_data['face_width4']
            bmi_data['WHR']=bmi_data['face_width4']/bmi_data['face_height2']                       
            bmi_data['PAR']=(seg_1+seg_2+seg_3+seg_4)/PolygonArea(corners)
            bmi_data['ES']=(bmi_data['eye_width1']-bmi_data['eye_width2'])/2.0 
            bmi_data['LF_FH']=bmi_data['face_height']/bmi_data['face_height3']
            bmi_data['FW_LFH']=bmi_data['face_width0']/bmi_data['face_height']
            bmi_data['MEH']=(d1+d2+d3+d4+d5+d6)/6.0                                                                                            
            columns2=['CJWR', 'WHR', 'PAR', 'ES', 'LF_FH', 'FW_LFH', 'MEH']   
            
            testing_inputs2=bmi_data[columns2].values
            testing_inputs2=np.array(testing_inputs2)
            testing_inputs2= sc2.transform(testing_inputs2)
            pred = clf2.predict(testing_inputs2)
            df4_neutral.loc[tra]=[Names[tra], BMI[tra], pred]
            
df4_neutral.loc[(df4_neutral['Prediction'] ==0),'Prediction'] = 'Underweight (16-18.5)'
df4_neutral.loc[(df4_neutral['Prediction'] ==1),'Prediction'] = 'Normal (18.5-25)'
df4_neutral.loc[(df4_neutral['Prediction'] ==2),'Prediction'] = 'Overweight (25-30)'
df4_neutral.loc[(df4_neutral['Prediction'] ==3),'Prediction'] = 'Moderately obese (30-35)'
df4_neutral.loc[(df4_neutral['Prediction'] ==4),'Prediction'] = 'Severely obese (35-40)'

/home/lpolania/enter/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [4]:
df4_neutral

,Athlete,Ground truth (BMI),Prediction
0,LIM HYOJUN,22.67,Normal (18.5-25)
1,DAVID GLEIRSCHER,26.87,Overweight (25-30)
